In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

In [2]:

loader = PyPDFLoader("../données/ComptesRendus/Cahier des charges Compte-rendu activité.pdf")
docs = loader.load()
print(type(docs))
print(docs)

<class 'list'>
[Document(metadata={'producer': 'macOS Version 14.6.1 (assemblage 23G93) Quartz PDFContext', 'creator': 'ONLYOFFICE', 'creationdate': "D:20241017100718Z00'00'", 'title': 'Sans titre', 'moddate': "D:20241017100718Z00'00'", 'source': '../données/ComptesRendus/Cahier des charges Compte-rendu activité.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content=''), Document(metadata={'producer': 'macOS Version 14.6.1 (assemblage 23G93) Quartz PDFContext', 'creator': 'ONLYOFFICE', 'creationdate': "D:20241017100718Z00'00'", 'title': 'Sans titre', 'moddate': "D:20241017100718Z00'00'", 'source': '../données/ComptesRendus/Cahier des charges Compte-rendu activité.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content='')]


In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader

def load_all_pdfs_from_folder(folder_path):
    documents = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                loader = PyPDFLoader(pdf_path)
                # Each file as a single document
                doc = loader.load()
                documents.append(doc)
    return documents

In [4]:
documents_comptes_rendus = load_all_pdfs_from_folder("../données/Hackaton_donnees")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong poin

In [ ]:
import os

# Count the number of PDF files in the specified folder and the number of documents loaded

folder_path = "../données/Hackaton_donnees"
num_files = sum(
    1 for root, _, files in os.walk(folder_path)
    for file in files if file.lower().endswith('.pdf')
)
print(f"Number of PDF files in folder: {num_files}")
print(f"Number of documents in documents_comptes_rendus: {len(documents_comptes_rendus)}")

Number of PDF files in folder: 436
Number of documents in documents_comptes_rendus: 436


In [7]:
for page in documents_comptes_rendus[0]:
    print(page.page_content)
# print(documents_comptes_rendus[0])

Roux Compte rendu d’activité 
Manon Écrit+ 
 
 
Introduction  
 
L’écrit occupe une place importante dans notre quotidien notamment quand nous 
sommes étudiants. Dans le cadre de ma formation, les professeurs nous demandent 
une rédaction propre, soignée et appliquée, que ce soit dans des dissertations ou 
des rapports. L’UEO certification Écrit+ était, selon moi, une opportunité pour prendre 
davantage confiance dans mes écrits. 
 
Je pense avoir une bonne maîtrise de l’écrit mais je sais également que dans le 
cadre académique, les professeurs exigent une maîtrise accrue de la langue. Ce bon 
niveau de français à l’écrit est également demandé dans le cadre professionnel et 
certains employeurs peuvent réclamer une certification de français.  
Ayant déjà obtenu une certification comme celle-ci (Certification Voltaire) il y a 
quelques années, Écrit+ me permettait de renforcer mes compétences.  
De plus, le semestre dernier j’ai pris cette UEO mais niveau 1 donc je souhaitais 
améliore

In [8]:
llm = ChatOllama(model='deepseek-r1:latest')

In [9]:
source = documents_comptes_rendus[0][0].metadata.get('source', 'unknown')
print(f"Source: {source}")

Source: ../données/Hackaton_donnees/S2 travaux étudiants/S2 travaux étudiants/ueo preparation a la certification ecri+ niveau 2  approfondissement (T-U12-9234) - Semestre 2-Mon cahier dactivité écri+ 24-25 au semestre pair (texte à déposer)-57942/Manon Roux Écrit+ copie.pdf


In [34]:
prompt = PromptTemplate.from_template(
    """ Tu dois répondre à la tâche suivante en te basant sur les documents suivants et la grille de réfléxivité, en suivant le format attendu écrit entre le DEBUT et la FIN de la sortie structurée.

    Tâche : Tu es un expert en sciences de l’éducation. À partir d’un texte rédigé par un étudiant, tu dois : Identifier les catégories de réflexivité présentes parmi 12 possibles. Attribuer un score global sur 10, selon la profondeur et la diversité de la réflexivité. Citer au moins 2 ou 3 extraits du texte, en les associant clairement aux catégories détectées.

    # Grille des 12 catégories de réflexivité : 
    | Niveau | Catégories| Signes distinctifs|

| 1      | **Narrer/décrire**                        | Récit simple de ce qui a été fait.            |
| 2      | **Questionner**                           | Doutes, interrogations, remises en question.  |
| 2      | **Prendre conscience**                    | Réalisations personnelles.                    |
| 2      | **Repérer ses difficultés**               | Reconnaissance d’obstacles ou limites.        |
| 3      | **Légitimer par préférence ou tradition** | Habitudes, valeurs personnelles.              |
| 3      | **Légitimer par contexte**                | Référence à l’environnement, aux contraintes. |
| 4      | **Légitimer par théorie ou éthique**      | Appui sur des auteurs, principes éthiques.    |
| 4      | **Pratique intentionnelle**               | Choix argumentés et assumés.                  |
| 5      | **Diagnostiquer**                         | Analyse de causes, compréhension globale.     |
| 5      | **Proposer des alternatives**             | Idées de solutions ou d’ajustements.          |
| 5      | **Explorer des alternatives**             | Comparaison, ouverture à d'autres pratiques.  |
| 6      | **Théoriser**                             | Construction de principes ou modèles.         |

# Format de sortie structuré attendu :
texte JSON avec les catégories suivantes :
# DEBUT DE SORTIE STRUCTURÉE

- Texte analysé : Nom  de famille de l'étudiant extrait de {source}

- Score réflexivité (1 à 6) selon la table de niveaux et catégories ci-dessus

- Année d’étude : L1, L2, L3 ou NODATA (si absent)

- "Catégories repérées" issue du tableau de niveaux et catégories ci-dessus

- "Extraits justificatifs" correspondant aux catégories détectées. exemples de structure :
    - [« Je me suis rendu compte que je ne laissais pas assez de temps de parole… »] → Prendre conscience
    - [« J’ai choisi cette approche car elle est courante dans mon établissement… »] → Légitimer par contexte
    - [« Cela rejoint ce que Meirieu souligne sur le sens en formation… »] → Légitimer par théorie
    (...)


# FIN DE SORTIE STRUCTURÉE

Réalise maintenant cette tâche en te basant sur le texte du document suivant :
\n\n{context}\n\n 

"""
)

In [35]:
chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

In [36]:
response =  chain.invoke({
    "context":documents_comptes_rendus[0],
    "source": source
})

In [37]:
print(response)

<think>
Bon, je dois analyser ce compte rendu d'activité d'un étudiant en Écrit+ pour identifier les catégories de réflexivité selon la grille fournie. Comme expert en sciences de l'éducation, je vais examiner chaque niveau et trouver des correspondances dans le texte.

D'abord, regardons l'introduction : l'étudiante mentionne qu'elle pensait avoir une bonne maîtrise de l'écrit mais reconnaît que les exigences académiques sont plus élevées. C'est un signe clair de **"Prendre conscience"** (niveau 2). Elle réalise qu'elle a des points faibles mal définis avant la certification.

Ensuite, elle décrit ses attentes initiales et note ses difficultés à mettre des mots sur ses forces/faiblesses. Cela montre une tentative de **"Diagnostiquer"** (niveau 5) et d'**"Explorer des alternatives"** (niveaux 5), puisqu'elle essaie différentes approches pour identifier les problèmes.

Dans la partie sur les séances, elle détaille ses progrès et recule : par exemple, à la séance 1, elle maîtrise 70% mai

In [38]:
import re

# Remove text between </think> and the next code block (```json)
response_cleaned = re.sub(r"<think>.*?</think>\n?", "", response, flags=re.DOTALL).strip()
print(response_cleaned)

```json
{
  "Texte analysé": "Roux Compte rendu d’activité Manon Écrit+",
  "Score réflexivité": 4,
  "Année d’étude": "L3",
  "Catégories repérées": [
    {
      "Niveau": 2,
      "Nom de catégorie": "**Prendre conscience**"
    },
    {
      "Niveau": 3,
      "Nom de catégorie": "**Légitimer par contexte**"
    },
    {
      "Niveau": 4,
      "Nom de catégorie": "**Pratique intentionnelle**"
    },
    {
      "Niveau": 5,
      "Nom de catégorie": "**Diagnostiquer**"
    }
  ],
  "Extraits justificatifs": [
    "[« Je pense avoir une bonne maîtrise de l’écrit mais je sais également que dans le cadre académique, les professeurs exigent une maîtrise accrue de la langue. »] → Prendre conscience (je réalise mes propres limites et celles des attentes scolaires)",
    "[« Ayant déjà obtenu une certification comme celle-ci (Certification Voltaire) il y a quelques années, Écrit+ me permettait de renforcer mes compétences. »] → Légitimer par contexte (référence à mon parcours précédent

In [39]:
import json
import pandas as pd
import re

# Extract JSON part from the response string
json_match = re.search(r'```json\n(.*?)\n```', response, re.DOTALL)
if json_match:
    json_str = json_match.group(1)
    data = json.loads(json_str)
    # Flatten the dictionary for DataFrame
    flat_data = {**data}
    df = pd.DataFrame([flat_data])
    print(df)
else:
    print("No JSON found in response.")

                               Texte analysé  Score réflexivité Année d’étude  \
0  Roux Compte rendu d’activité Manon Écrit+                  4            L3   

                                 Catégories repérées  \
0  [{'Niveau': 2, 'Nom de catégorie': '**Prendre ...   

                              Extraits justificatifs  
0  [[« Je pense avoir une bonne maîtrise de l’écr...  


In [40]:
df


,Texte analysé,Score réflexivité,Année d’étude,Catégories repérées,Extraits justificatifs
0,Roux Compte rendu d’activité Manon Écrit+,4,L3,"[{'Niveau': 2, 'Nom de catégorie': '**Prendre ...",[[« Je pense avoir une bonne maîtrise de l’écr...
